# Llama Stack Responses API with MCP Tools

This notebook demonstrates using Llama Stack's native MCP (Model Context Protocol) integration.

## Configuration

This notebook uses environment variables from `.env` file in the project root.
Create your own `.env` file based on `.env.example`.

In [25]:
!pip3 install llama-stack-client==0.2.22


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [ ]:
# Load environment variables
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Load .env from project root
project_root = Path.cwd().parent.parent if 'assets/notebooks' in str(Path.cwd()) else Path.cwd()
env_path = project_root / '.env'
load_dotenv(env_path)

print(f"📁 Loading environment from: {env_path}")
print(f"✅ .env file {'found' if env_path.exists() else 'NOT FOUND'}")

# Verify kernel and version
import llama_stack_client

print(f"\n🐍 Python: {sys.executable}")
print(f"📦 llama-stack-client: {llama_stack_client.__version__}")
print(f"📋 Expected version: 0.2.22")

# Check if using correct venv
if ".venv" in sys.executable:
    print("✅ Using project venv - CORRECT!")
else:
    print("⚠️  Using global Python - Consider switching kernel!")
    print("   Click 'Select Kernel' → Choose 'Python (byo-agentic-framework)'")

In [27]:
!pip3 list | grep llama

llama_stack_client        0.2.22


In [ ]:
from llama_stack_client import LlamaStackClient

# Get configuration from environment variables
LLAMA_STACK_BASE_URL = os.getenv("LLAMA_STACK_BASE_URL")
LLAMA_STACK_OPENAI_ENDPOINT = os.getenv("LLAMA_STACK_OPENAI_ENDPOINT")

print(f"🌐 Llama Stack Configuration:")
print(f"   Base URL: {LLAMA_STACK_BASE_URL}")
print(f"   OpenAI Endpoint: {LLAMA_STACK_OPENAI_ENDPOINT}")

# Create client
client = LlamaStackClient(base_url=LLAMA_STACK_OPENAI_ENDPOINT)
print(f"✅ Llama Stack client created")

In [29]:
import json
from datetime import date

def pretty_print(obj) -> None:
    """
    Recursively prints an object's __dict__ in a nicely formatted JSON.
    Handles nested objects and lists of objects.
    """
    def recursive_serializer(o):
        if hasattr(o, '__dict__'):
            return o.__dict__
        # Handle types that are not directly JSON serializable
        if isinstance(o, date):
            return o.isoformat()
        # For other types, raise a TypeError to let the default handler fail.
        raise TypeError(f"Object of type {o.__class__.__name__} is not JSON serializable")

    # Determine what to serialize
    data_to_serialize = obj.__dict__ if hasattr(obj, "__dict__") else obj

    print(json.dumps(
        data_to_serialize,
        indent=2,
        default=recursive_serializer
    ))

In [ ]:
# Get configuration from environment
INFERENCE_MODEL = os.getenv("INFERENCE_MODEL")
MCP_WEATHER_SERVER_URL = os.getenv("MCP_WEATHER_SERVER_URL")

NPS_EXAMPLE_PROMPT = "Tell me about some parks in Rhode Island"

print(f"🤖 Model: {INFERENCE_MODEL}")
print(f"🌦️  MCP Server: {MCP_WEATHER_SERVER_URL}")
print(f"💬 Prompt: {NPS_EXAMPLE_PROMPT}\n")

responses = client.responses.create(
    model=INFERENCE_MODEL,
    input=NPS_EXAMPLE_PROMPT,
    tools=[
        {
            "type": "mcp",
            "server_url": MCP_WEATHER_SERVER_URL,
            "server_label": "National Parks Service tools",
        }
    ]
)

pretty_print(responses)